In [14]:
import pandas as pd
import datetime

# 读取负荷数据
load_data = pd.read_excel('附件1：各园区典型日负荷数据.xlsx')
# 读取风光发电数据
generation_data = pd.read_excel('附件2_new.xlsx')
# 读取季节性风光发电数据
seasonal_generation_data = pd.read_excel('附件3：12个月各园区典型日风光发电数据.xlsx')

# 设置索引
load_data.set_index('时间（h）', inplace=True)
generation_data.set_index('时间（h）', inplace=True)
seasonal_generation_data.set_index('时间（h）', inplace=True)

# 定义分时电价表
time_of_day_prices = {
    'day': 1,
    'night': 0.4
}

# 计算购电量和弃风弃光电量
def calculate_purchase_and_abandonment(load_data, generation_data, park_name):
    load_col = f"Load_{park_name}"
    wind_col = f"{park_name}_Wind"
    solar_col = f"{park_name}_Solar"
    
    total_load = load_data[load_col].sum()
    total_wind_gen = generation_data[wind_col].sum()
    total_solar_gen = generation_data[solar_col].sum()
    total_gen = total_wind_gen + total_solar_gen
    
    purchase = max(0, total_load - total_gen)
    abandonment = max(0, total_gen - total_load)
    
    return purchase, abandonment

# 计算自发电成本
def calculate_self_generation_cost(generation_data, park_name, wind_cost_per_kwh=0.5, solar_cost_per_kwh=0.4):
    wind_col = f"{park_name}_Wind"
    solar_col = f"{park_name}_Solar"
    
    wind_gen = generation_data[wind_col].sum()
    solar_gen = generation_data[solar_col].sum()
    
    self_gen_cost = wind_gen * wind_cost_per_kwh + solar_gen * solar_cost_per_kwh
    
    return self_gen_cost

# 计算购电成本
def calculate_purchase_cost(purchase_data, time_of_day_prices):
    def get_price(hour):
        if 7 <= hour < 22:
            return time_of_day_prices['day']
        else:
            return time_of_day_prices['night']
    
    purchase_cost = purchase_data.index.to_series().apply(lambda x: get_price(x.hour) * purchase_data.loc[x])
    
    return purchase_cost.sum()


# 计算总供电成本
def calculate_total_supply_cost(self_gen_cost, purchase_cost):
    return self_gen_cost + purchase_cost

# 计算储能系统成本
def calculate_storage_cost(storage_capacity, storage_power_cost, storage_energy_cost):
    storage_cost = 0
    for park, data in storage_capacity.items():
        storage_cost += data['power'] * storage_power_cost
        storage_cost += data['energy'] * storage_energy_cost
    return storage_cost

# 计算装机容量
def calculate_capacity(load_data, generation_data):
    capacity = {}
    for park in load_data.columns:
        if 'Load' in park:
            park_name = park.split('_')[1]
            wind_col = f"{park_name}_Wind"
            solar_col = f"{park_name}_Solar"
            
            max_load = load_data[park].max()
            total_wind_gen = generation_data[wind_col].sum()
            total_solar_gen = generation_data[solar_col].sum()
            total_gen = total_wind_gen + total_solar_gen
            
            if total_gen == 0:
                wind_capacity = 0
                solar_capacity = 0
            else:
                avg_wind_gen = total_wind_gen / len(generation_data)
                avg_solar_gen = total_solar_gen / len(generation_data)
                
                wind_capacity = max_load / avg_wind_gen if avg_wind_gen != 0 else 0
                solar_capacity = max_load / avg_solar_gen if avg_solar_gen != 0 else 0
            
            capacity[park_name] = {
                'wind': wind_capacity,
                'solar': solar_capacity
            }
    return capacity

# 计算储能系统容量
def calculate_storage_capacity(load_data, storage_power_cost, storage_energy_cost, min_soc=0.1, max_soc=0.9):
    storage = {}
    for park in load_data.columns:
        if 'Load' in park:
            max_load = load_data[park].max()
            park_name = park.split('_')[1]
            storage_power = max_load * 0.8
            storage_energy = storage_power * (max_soc - min_soc)
            
            storage[park_name] = {
                'power': storage_power,
                'energy': storage_energy,
                'cost': storage_power * storage_power_cost + storage_energy * storage_energy_cost
            }
    return storage

# 定义单价
wind_cost_per_kw = 3000
solar_cost_per_kw = 2500
storage_power_cost = 800
storage_energy_cost = 1800

# 计算装机容量和储能系统容量
capacity = calculate_capacity(load_data, generation_data)
print("Capacity:", capacity)

storage_capacity = calculate_storage_capacity(load_data, storage_power_cost, storage_energy_cost)
print("Storage capacity:", storage_capacity)

# 各园区经济状况分析
parks = ['A', 'B', 'C']
results = {}

for park in parks:
    print(f"Calculating for Park {park}...")
    
    purchase, abandonment = calculate_purchase_and_abandonment(load_data, generation_data, park)
    print(f"Park {park} - Purchase: {purchase}, Abandonment: {abandonment}")
    
    self_gen_cost = calculate_self_generation_cost(seasonal_generation_data, park)
    print(f"Park {park} - Self Generation Cost: {self_gen_cost}元")
    
    purchase_data = pd.Series([purchase] * len(load_data), index=load_data.index)
    purchase_cost = calculate_purchase_cost(purchase_data, time_of_day_prices)
    print(f"Park {park} - Purchase Cost: {purchase_cost}元")
    
    total_supply_cost = calculate_total_supply_cost(self_gen_cost, purchase_cost)
    print(f"Park {park} - Total Supply Cost: {total_supply_cost}元")
    
    storage_cost = calculate_storage_cost({park: storage_capacity[park]}, storage_power_cost, storage_energy_cost)
    print(f"Park {park} - Storage Cost: {storage_cost}元")
    
    total_cost = total_supply_cost + storage_cost
    print(f"Park {park} - 总成本为：{total_cost}元")
    
    annualized_cost = total_cost / 5  # 假设投资回报期为5年
    print(f"Park {park} - 总年化成本为：{annualized_cost}元")
    
    results[park] = {
        'purchase': purchase,
        'abandonment': abandonment,
        'self_gen_cost': self_gen_cost,
        'purchase_cost': purchase_cost,
        'total_supply_cost': total_supply_cost,
        'storage_cost': storage_cost,
        'total_cost': total_cost,
        'annualized_cost': annualized_cost
    }

print("各园区经济状况分析结果:", results)

for park in parks:
    wind_capacity = capacity[park]['wind']
    solar_capacity = capacity[park]['solar']
    print(f'园区{park} 风电装机容量: {wind_capacity} kW')
    print(f'园区{park} 光伏装机容量: {solar_capacity} kW')


Capacity: {'A': {'wind': 0, 'solar': 2022.5863011632512}, 'B': {'wind': 1628.4492809949475, 'solar': 0}, 'C': {'wind': 2087.7817319098453, 'solar': 2210.491827150606}}
Storage capacity: {'A': {'power': 357.6, 'energy': 286.08000000000004, 'cost': 801024.0}, 'B': {'power': 335.20000000000005, 'energy': 268.16, 'cost': 750848.0000000001}, 'C': {'power': 404.8, 'energy': 323.84000000000003, 'cost': 906752.0}}
Calculating for Park A...
Park A - Purchase: 7895.6959, Abandonment: 0
Park A - Self Generation Cost: 0.48436000000000007元
Park A - Purchase Cost: 146859.94374元
Park A - Total Supply Cost: 146860.4281元
Park A - Storage Cost: 801024.0元
Park A - 总成本为：947884.4281元
Park A - 总年化成本为：189576.88562000002元
Calculating for Park B...
Park B - Purchase: 7703.8248, Abandonment: 0
Park B - Self Generation Cost: 3.0875000000000004元
Park B - Purchase Cost: 143291.14128元
Park B - Total Supply Cost: 143294.22878元
Park B - Storage Cost: 750848.0000000001元
Park B - 总成本为：894142.2287800001元
Park B - 总年化成本为